In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import os.path
import skimage
import skimage.color
import skimage.transform
import skimage.segmentation
import sklearn.preprocessing
import sklearn.model_selection
import math
import shutil
import pathlib
import glob
import shutil
import uuid
import random
import platform
import torch
import torchvision
import numpy as np
import scipy as sp
import scipy.io
import scipy.signal
import pandas as pd
import networkx
import wfdb
import json
import tqdm
import dill
import pickle
import matplotlib.pyplot as plt

import sak
import sak.hash
import sak.signal.wavelet
import sak.data
import sak.data.augmentation
import sak.visualization
import sak.torch
import sak.torch.nn
import sak.torch.nn as nn
import sak.torch.train
import sak.torch.data
import sak.data.preprocessing
import sak.torch.models
import sak.torch.models.lego
import sak.torch.models.variational
import sak.torch.models.classification


In [3]:
basedir = '/media/guille/DADES/DADES/Delineator'
model_name = 'WNet5LevelsSelfAttentionDiceOnly_20201130125349'
valid_folds = sak.load_data(os.path.join(basedir,'TrainedModels',model_name,'validation_files.csv'),dtype=None)

basepath = os.path.join(basedir,'TrainedModels',model_name)

# Save state dict

In [5]:
for i in tqdm.tqdm(range(5)):
    for model_type in ["model_best", "checkpoint"]:
        if os.path.isfile(os.path.join(basepath,f'fold_{i+1}',f'{model_type}_unconverted.model')):
            model = torch.load(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.model'),pickle_module=dill)
            
            state_dict = model.state_dict()
            with open(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.modeldict'), "wb") as f:
                dill.dump(state_dict,f)
        elif os.path.isfile(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.model')):
            model = torch.load(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.model'),pickle_module=dill)
            
            state_dict = model.state_dict()
            with open(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.modeldict'), "wb") as f:
                dill.dump(state_dict,f)
                
            shutil.copy(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.model'),
                        os.path.join(basepath,f'fold_{i+1}',f'{model_type}_unconverted.model'))
        else:
            print("File for fold {} not found. Continuing...".format(i+1))

100%|██████████| 5/5 [00:05<00:00,  1.12s/it]


# Change Kernel & Load state dict

In [5]:
config = sak.load_config(os.path.join(basepath,f"{'_'.join(model_name.split('_')[:-1])}.json"))

In [9]:
for i in tqdm.tqdm(range(5)):
    for model_type in ["model_best", "checkpoint"]:
        if os.path.isfile(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.modeldict')):
            with open(os.path.join(basepath,f'fold_{i+1}',f'{model_type}.modeldict'), "rb") as f:
                # Load state dict
                state_dict = dill.load(f)
                
            # Define model
            try:
                model = nn.ModelGraph(
                    config["model"]["functions"],
                    config["model"]["nodes"],
                    config["model"]["edges"],
                )
            except:
                model = sak.from_dict(config["model"])
            
            model = model.float().cuda()
                
            # Match keys
            model.load_state_dict(state_dict)
            
            # Save model
            torch.save(model,os.path.join(basepath,f'fold_{i+1}',f'{model_type}.model'),pickle_module=dill)
        else:
            print("File for fold {} not found. Continuing...".format(i+1))

100%|██████████| 5/5 [00:02<00:00,  1.76it/s]
